In [88]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tck
import pandas as pd
import sympy as sym
import warnings
import pickle as plk
from IPython.display import display, Math
from numpy.lib.scimath import sqrt as csqrt
from scipy.signal import find_peaks

import os
import imp
import sys
import importlib.util

###############################################################
# LINUX PATH

sys.path.append("/opt/lumerical/v221/api/python") #Default linux lumapi path
import lumapi
sys.path.append(os.path.dirname('/home/trainner/Documentos/Caio/GitReps/Photonics-training/Projects/Optical_Hybrid/SimulationFiles')) #Current directory

###############################################################
# WINDOWS PATH

# spec_win = importlib.util.spec_from_file_location('lumapi', 'C:\\Program Files\\Lumerical\\v202\\api\\python\\lumapi.py')
# lumapi = importlib.util.module_from_spec(spec_win)
# os.add_dll_directory("C:/Program Files/Lumerical/v202/api/python")
# sys.path.append(os.path.dirname('C:/Users/Caio/Documents/GitHub/Photonics-training/Projects/Optical_Hybrid/SimulationFiles/'))
# spec_win.loader.exec_module(lumapi)

In [89]:
hyb = lumapi.FDTD(filename='Hybrid.fsp',hide=False) 

## Functions

In [90]:
# Search for function's intersections on signal levels
def signalLv(yarray,level,type):
    peak, pd = find_peaks(yarray)
    maxpeak = yarray[peak[np.argmax(yarray[peak])]]
    minpeak = yarray[np.argmin(yarray)]
    index = []

    if(type == 'linear'):
        lvpeak = (maxpeak - minpeak)*level + minpeak
        for i in range(0,np.size(yarray)-1):
            if(((maxpeak-yarray[i]) - (maxpeak-lvpeak)) * ((maxpeak-yarray[i+1]) - (maxpeak-lvpeak)) <= 0):
                index.append(i)
        yMed = (yarray[index[0]] + yarray[index[-1]])/2

    if(type == 'log10'):
        lvpeak = maxpeak + 10*np.log10(level)
        for i in range(0,np.size(yarray)-1):
            if(((maxpeak-yarray[i]) - (maxpeak-lvpeak)) * ((maxpeak-yarray[i+1]) - (maxpeak-lvpeak)) <= 0):
                index.append(i)
        yMed = 10*np.log10((10**(yarray[index[0]]/2) + 10**(yarray[index[1]]/2))/2)

    return {'index':index,'yMed':yMed}


# Creates a box
def RectInsert(X,XSPAN,Y,YSPAN,Z,ZSPAN,MATERIAL,lumerical):
    
    lumerical.switchtolayout()
    lumerical.addrect()
    lumerical.set({"x":X,"y":Y,"z":Z,
         "x span":XSPAN, "y span":YSPAN, "z span":ZSPAN,
         "material":MATERIAL
        })


# Format a function/dataset as a polygon
def PolyForm(x,y,ybase):
    y2 = y + ybase
    A = np.zeros((2,len(x)+2))
    
    A[0][1:len(x)+1] = x
    A[1][1:len(x)+1] = y2
    A[0][0] = x[0]
    A[0][len(x)+1] = x[-1]
    A[1][0] = ybase
    A[1][len(x)+1] = ybase
    
    return A

## Assembly parameters

In [120]:
# Topology ------------------------------------
InnerBd = 2* 1.6e-6  # Inner body X span (Sum of both parts)
OuterBd = 2* 2.4e-6  # Outer body X span (Sum of both parts)

CentWid = 1.48e-6  # Central Width
InnerWid = 1.48e-6  # Inner Width
OuterWid = 1.6e-6  # Outer Width

TLen = 1e-6  # Tapper length

Material = "Si (Silicon) - Palik"

SXsp = 5e-6  # S band xspan
SYsp = 2.5e-6  # S band yspan


# Simulation ---------------------------------
FDTDAccuracy = 4
simTime = 4000e-15

## Assembly

In [151]:
hyb.switchtolayout()
hyb.deleteall()

# Creating the Full Body
Verts = np.array([[0,CentWid/2], [InnerBd/2,InnerWid/2], [(InnerBd+OuterBd)/2,OuterWid/2],
         [(InnerBd+OuterBd)/2,-OuterWid/2], [InnerBd/2,-InnerWid/2], [0,-CentWid/2],
         [-InnerBd/2,-InnerWid/2], [-(InnerBd+OuterBd)/2,-OuterWid/2],
         [-(InnerBd+OuterBd)/2,OuterWid/2], [-InnerBd/2,InnerWid/2]])   # Vertices

Halflen = (InnerBd+OuterBd)/2 # MMI half length

MMIpos = {"x":[-4e-6 - Halflen-TLen-0.45e-6, 0 , 4e-6 + Halflen+TLen+0.45e-6],
          "y":[0,-4e-6 - Halflen-TLen-0.45e-6, 0],
          "angle":[0,90,180]}

GenPar = {"first axis":"z", "z":0,
         "len":TLen, "thickness":0.22e-6,
         "width_r":0.7e-6, "width_l":0.5e-6, 
         "angle_side":90}    # General parameter sets

GenPar2 = {"base angle":90, "base height":0.22e-6, "base width":0.5e-6,
           "x span":SXsp, "y span":SYsp, "first axis":"y", "material":Material}    # General parameter sets

# Creating MMIs and tappers
for i in range(3):
        hyb.addpoly()
        hyb.set("name", "FullBody " + str(i+1))
        hyb.set("vertices",Verts)
        hyb.set({"z span":220e-9, "material":Material})
        hyb.set({"x":0, "y":0, "z":0})
        hyb.addtogroup("MMI " + str(i+1))

        hyb.addobject("linear_taper")
        hyb.set(GenPar)
        hyb.set({"rotation 1":0, "name":"Input Taper 1",
                "x":-TLen/2-(InnerBd+OuterBd)/2, "y":OuterWid/2 - 0.35e-6})

        hyb.addobject("linear_taper")
        hyb.set(GenPar)
        hyb.set({"rotation 1":0, "name":"Input Taper 2",
                "x":-TLen/2-(InnerBd+OuterBd)/2, "y":-OuterWid/2 + 0.35e-6})

        hyb.addobject("linear_taper")
        hyb.set(GenPar)
        hyb.set({"rotation 1":180, "name":"Output Taper 1",
                "x":TLen/2+(InnerBd+OuterBd)/2, "y":OuterWid/2 - 0.35e-6})

        hyb.addobject("linear_taper")
        hyb.set(GenPar)
        hyb.set({"rotation 1":180, "name":"Output Taper 2",
                "x":TLen/2+(InnerBd+OuterBd)/2, "y":-OuterWid/2 + 0.35e-6})

        RectInsert(-TLen/2-(InnerBd+OuterBd)/2 - 1e-6 ,1e-6,
                   OuterWid/2 - 0.35e-6,0.5e-6,0,220e-9,Material,hyb)
        hyb.addtogroup("MMI " + str(i+1))

        RectInsert(-TLen/2-(InnerBd+OuterBd)/2 - 1e-6 ,1e-6,
                   -OuterWid/2 + 0.35e-6 ,0.5e-6,0,220e-9,Material,hyb)
        hyb.addtogroup("MMI " + str(i+1))

        hyb.addobject('s_bend_wg')
        hyb.set(GenPar2)
        hyb.set({"name":"Input bend 1", "rotation 1":180,
                "x":-TLen/2-(InnerBd+OuterBd)/2 - 1.5e-6, "y":0.45e-6, "z":0})

        hyb.addobject('s_bend_wg')
        hyb.set(GenPar2)
        hyb.set({"name":"Input bend 2", "rotation 1":0,
                "x":-SXsp -TLen/2-(InnerBd+OuterBd)/2 - 1.5e-6, "y":-0.45e-6-SYsp, "z":0})

        RectInsert(-TLen/2-(InnerBd+OuterBd)/2 - 2e-6-SXsp ,1e-6,
                   OuterWid/2 - 1.25e-6-SYsp,0.5e-6,0,220e-9,Material,hyb)
        hyb.addtogroup("MMI " + str(i+1))

        RectInsert(-TLen/2-(InnerBd+OuterBd)/2 - 2e-6-SXsp ,1e-6,
                   -OuterWid/2 + 1.25e-6+SYsp ,0.5e-6,0,220e-9,Material,hyb)
        hyb.addtogroup("MMI " + str(i+1))

        hyb.select("MMI " + str(i+1))
        hyb.set({"x":MMIpos["x"][i], "y":MMIpos["y"][i], "z":0, "first axis":"z"})
        hyb.set("rotation 1",MMIpos["angle"][i])


# Creating Y-Branch (LO entrance)
Verts = np.array([[-1.35e-6, 0.5e-6/2],[-1.166e-6, 0.55e-6/2],
                  [-1e-6, 0.62e-6/2], [-0.833e-6, 0.7e-6/2], [-0.666e-6, 0.9e-6/2],
                  [-0.499e-6, 1.26e-6/2], [-0.332e-6, 1.41e-6/2], [-0.165e-6, 1.55e-6/2],
                  [0.002e-6, 1.63e-6/2], [0.169e-6, 1.63e-6/2], [0.336e-6, 1.63e-6/2],
                  [0.503e-6, 1.62e-6/2], [1e-6, 1.6e-6/2],
                  
                  [1e-6, -1.6e-6/2],
                  [0.503e-6, -1.62e-6/2], [0.336e-6, -1.63e-6/2], [0.169e-6, -1.63e-6/2],
                  [0.002e-6, -1.63e-6/2], [-0.165e-6, -1.55e-6/2], [-0.332e-6, -1.41e-6/2],
                  [-0.499e-6, -1.26e-6/2], [-0.666e-6, -0.9e-6/2], [-0.833e-6, -0.7e-6/2],
                  [-1e-6, -0.62e-6/2],[-1.166e-6, -0.55e-6/2],[-1.35e-6, -0.5e-6/2]])   # Vertices
  
hyb.addpoly()
hyb.set("name", "FullBody")
hyb.set("vertices",Verts)
hyb.set({"x":0, "y":0, "z":0, "z span":220e-9,
        "material":Material})
hyb.addtogroup("LO Input")

RectInsert(-1.85e-6,1e-6,0,0.5e-6,0,220e-9,Material,hyb)
hyb.addtogroup("LO Input")

GenPar = {"first axis":"z", "z":0,
         "len":TLen, "thickness":0.22e-6,
         "width_r":0.5e-6, "width_l":0.7e-6, 
         "angle_side":90}    # General parameter sets

hyb.addobject("linear_taper")
hyb.set(GenPar)
hyb.set({"rotation 1":0, "name":"Output Taper 1",
         "x":(2e-6+TLen)/2, "y":0.45e-6})

hyb.addobject("linear_taper")
hyb.set(GenPar)
hyb.set({"rotation 1":0, "name":"Output Taper 2",
         "x":(2e-6+TLen)/2, "y":-0.45e-6})

hyb.select("LO Input")
hyb.set("first axis","z")
hyb.set("rotation 1",270)
hyb.set({"x":0, "y":6.45e-6, "z":0})


# Bends and post-tappers
GenPar = {"base angle":90, "base height":0.22e-6, "base width":0.5e-6,
                 "radius":4e-6, "material":Material}    # General parameter sets

hyb.addobject('90_bend_wg')
hyb.set(GenPar)
hyb.set({"start angle":270, "name":"Routing bend 2",
        "x":MMIpos["x"][0] + Halflen+TLen, "y":4e-6 + 0.45e-6, "z":0})

hyb.addobject('90_bend_wg')
hyb.set(GenPar)
hyb.set({"start angle":0, "name":"Routing bend 1",
        "x":MMIpos["x"][0] + Halflen+TLen, "y":- 4e-6 - 0.45e-6, "z":0})

hyb.addobject('90_bend_wg')
hyb.set(GenPar)
hyb.set({"start angle":180, "name":"Routing bend 3",
        "x":MMIpos["x"][-1] - Halflen-TLen, "y":4e-6 + 0.45e-6, "z":0})

hyb.addobject('90_bend_wg')
hyb.set(GenPar)
hyb.set({"start angle":90, "name":"Routing bend 4",
        "x":MMIpos["x"][-1] - Halflen-TLen, "y":- 4e-6 - 0.45e-6, "z":0})



# FDTD Setting
hyb.addfdtd()
hyb.set({"x":0, "y":-4e-6-SYsp, "z":0,
         "z span":3e-6,
         "simulation time":simTime, "mesh accuracy":FDTDAccuracy,
         "z min bc":"Symmetric"})
hyb.set({"x span":5*Halflen + 13e-6 + 2*SXsp, "y span": 4*Halflen+10e-6 + SXsp})
hyb.set("background material","SiO2 (Glass) - Palik")


# Creating Ports
hyb.addport()
hyb.set({"x":-3*Halflen -4e-6 -SXsp, "y":0.45e-6+SYsp, "z":0,
        "y span":3e-6, "z span":3e-6,
        "direction":"Forward"})

hyb.addport()
hyb.set({"x":-3*Halflen -4e-6 -SXsp, "y": -0.45e-6-SYsp, "z":0,
        "y span":3e-6, "z span":3e-6,
        "direction":"Forward"})

hyb.addport()
hyb.set({"x":3*Halflen +4e-6 +SXsp, "y":0.45e-6+SYsp, "z":0,
        "y span":3e-6, "z span":3e-6,
        "direction":"Backward"})

hyb.addport()
hyb.set({"x":3*Halflen +4e-6 +SXsp, "y": -0.45e-6-SYsp, "z":0,
        "y span":3e-6, "z span":3e-6,
        "direction":"Backward"})

hyb.addport()
hyb.set("injection axis","y-axis")
hyb.set({"x":-0.45e-6-SYsp, "y": -3*Halflen -4e-6 -SXsp, "z":0,
        "x span":3e-6, "z span":3e-6,
        "direction":"Forward"})

hyb.addport()
hyb.set("injection axis","y-axis")
hyb.set({"x":0, "y": 7.9e-6, "z":0,
        "x span":3e-6, "z span":3e-6,
        "direction":"Backward"})

# Editing source settings
hyb.select('FDTD::ports')
hyb.set('monitor frequency points',1000)
hyb.setglobalsource("wavelength start", 1530e-9)
hyb.setglobalsource("wavelength stop", 1565e-9)


# Creating monitor region
hyb.addpower()  # Planar monitor
hyb.set({"x":0, "x span":5*Halflen + 12e-6, "y":-4e-6, "y span":4*Halflen+8e-6})
hyb.setglobalmonitor("use source limits",0)
hyb.setglobalmonitor("minimum wavelength",1530e-9)
hyb.setglobalmonitor("maximum wavelength",1565e-9)
hyb.setglobalmonitor("frequency points",15)

1.0